In [7]:
# import necessary libraries

import warnings
warnings.filterwarnings(action='ignore')

import numpy as np
import pandas as pd

from evidently import Report
from evidently.presets import DataDriftPreset

In [8]:
# Read the dataset
ref_data = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')  # Load reference data

# Create current data by modifying 'MonthlyCharges' with some noise
curr_data = ref_data.copy()
noise = np.random.normal(0, 5, size=len(curr_data))
curr_data['MonthlyCharges'] = (curr_data['MonthlyCharges'] * 1.2) + 10 + noise
indices_to_change = curr_data.sample(frac=0.5, random_state=42).index
curr_data.loc[indices_to_change, 'Contract'] = 'Month-to-month'

In [9]:
def generate_drift_report(ref_data, curr_data, output_file="drift_report.html"):

    drift_report = Report(metrics=[
        DataDriftPreset()
    ])

    eval = drift_report.run(reference_data=ref_data, current_data=curr_data)
    eval.save_html(output_file)
    eval = eval.dict()

    cleaned_data = []

    for item in eval['metrics']:
        if 'column' in item['config']:
            column = item['config']['column']
            method = item['config']['method']
            score = item['value']
            threshold = item['config']['threshold']
            
            if score > threshold:
                cleaned_data.append({
                    'Column': column,
                    'Drift Score': round(score, 4), 
                    'Threshold': threshold,
                    'Method': method
                })

    df_result = pd.DataFrame(cleaned_data)
    print(f"Drift report saved to {output_file}")

    if (df_result.empty):
        print("No drifted columns detected.")
    else:
        print("Drifted columns detected:")
        print(df_result)

# Generate the drift report
generate_drift_report(ref_data, curr_data)

Drift report saved to drift_report.html
Drifted columns detected:
           Column  Drift Score  Threshold                         Method
0  MonthlyCharges       0.7649        0.1  Wasserstein distance (normed)
1        Contract       0.1651        0.1        Jensen-Shannon distance


In [ ]:
def raw_preprocessor(df):
    df = df.copy()
    if 'customerID' in df.columns:
        df = df.drop('customerID', axis=1)
    if 'TotalCharges' in df.columns:
        df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
        df['TotalCharges'].fillna(df['TotalCharges'].median(), inplace=True)    
    if 'SeniorCitizen' in df.columns:   
        df['SeniorCitizen'] = df['SeniorCitizen'].replace({0: 'No', 1: 'Yes'})
    # if 'Churn' in df.columns:
    #         df['Churn'] = df['Churn'].replace({'No': 0, 'Yes': 1})
    if 'tenure' in df.columns:
        bins = [0, 6, 12, 18, 24, 30, 36, 42, 48, 54, 60, 66, 72]
        labels = ['0-0.5 Year', '0.5-1 Year', '1-1.5 Years', '1.5-2 Years', '2-2.5 Years', '2.5-3 Years', '3-3.5 Years', '3.5-4 Years', '4-4.5 Years', '4.5-5 Years', '5-5.5 Years', '5.5-6 Years']
        df['TenureGroup'] = pd.cut(df['tenure'], bins=bins, labels=labels, right=False)
        df.drop('tenure', axis=1, inplace=True)
    return df

raw_preprocessor = FunctionTransformer(raw_preprocessor)

In [ ]:
cat_pipeline = make_pipeline(
    SimpleImputer(strategy='most_frequent'),
    OneHotEncoder(handle_unknown='ignore')
)

num_pipeline = make_pipeline(
    SimpleImputer(strategy='median'),
    StandardScaler()
)

log_pipeline = make_pipeline(
    SimpleImputer(strategy='median'),
    FunctionTransformer(np.log1p, feature_names_out="one-to-one"),
    StandardScaler()
)

In [ ]:
cat_features = train_X.select_dtypes(include=['object']).columns.tolist()
num_features = train_X.select_dtypes(include=['int64', 'float64']).columns.tolist()

preprocessor = ColumnTransformer([
    ("cat", cat_pipeline, cat_features),
    ("log", log_pipeline, ['MonthlyCharges', 'TotalCharges'])
])

In [ ]:
model = best_model  # Assume best_model is defined elsewhere

In [ ]:
classification_pipeline = Pipeline([
    ("raw_fix", raw_preprocessor),
    ("preprocessor", preprocessor),
    ("classifier", xbgb_clf)
])

In [ ]:
classification_pipeline.predict(new_data)